# Faults sources within an area source
Given an area source and a set of faults we find the faults whose trace intersects the polygon of the area source and computes the percentage of the fault trace inside the polygon.

%%html
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<p style="font-size:60%;">
<a href="javascript:code_toggle()">[Toggle Code]</a>
<a target="_blank" href="./../project/project_set_params_gui.ipynb#">[Set params]</a
</p>

In [1]:
%matplotlib inline
import os
import re
import sys
import numpy
#from prettytable import PrettyTable
from oqmbt.oqt_project import OQtProject
from oqmbt.tools.geo import get_idx_points_inside_polygon, get_line_inside_polygon

In [2]:
prj_path = "/Users/kjohnson/GEM/Regions/paisl18/project/paisl.oqmbtp"
os.environ["OQMBT_PROJECT"] = prj_path

## Info

In [3]:
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
print('Reading project file: %s' % (project_pickle_filename))
oqtkp = OQtProject.load_from_file(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
try:
    area_source_ids_list = getattr(oqtkp,'active_source_id')
except:
    print('Active source ID not defined in the OQMBT project')
    area_source_ids_list = ['10']
print('Processing area source with ID:', area_source_ids_list[0])

Reading project file: /Users/kjohnson/GEM/Regions/paisl18/project/paisl.oqmbtp
Processing area source with ID: 16


## Selected faults

In [4]:
idx_faults_in_polygon = []
frac_faults_in_polygon = {}


In [6]:
assert len(area_source_ids_list) == 1
area_source = model.sources[area_source_ids_list[0]]
area_source.ids_faults_inside = {}
#
# Processing faults
for key in model.sources:
    fault = model.sources[key]
    if fault.source_type == 'SimpleFaultSource':
        plons = []
        plats = []
        pdeps = []
        #
        # Create lists with the coordinates of the points defining the trace
        for pnt in fault.trace.points:
            plons.append(pnt.longitude)
            plats.append(pnt.latitude)
            pdeps.append(pnt.depth)

        # Find points on the trace within the polygon
        iii = get_idx_points_inside_polygon(plons, 
                                            plats, 
                                            area_source.polygon.lons, 
                                            area_source.polygon.lats, 
                                            numpy.arange(0, len(plons)+1))
        #
        #
        frac = get_line_inside_polygon(plons, 
                                       plats, 
                                       area_source.polygon.lons, 
                                       area_source.polygon.lats)
        #
        #
        if frac is not None and frac > 0.0 and hasattr(fault, 'slip_rate'):
            idx_faults_in_polygon.append(fault.source_id)
            frac_faults_in_polygon[fault.source_id] = frac
            recurrence = fault.recurrence if hasattr(fault, 'recurrence') else None
            slip_rate = fault.slip_rate if hasattr(fault, 'slip_rate') else None
            #p.add_row([fault.source_id, fault.name, frac, slip_rate, recurrence])
            #
            # updating information for the current area source
            if hasattr(area_source, 'ids_faults_inside'):
                area_source.ids_faults_inside[fault.source_id] = frac
            else:
                area_source.ids_faults_inside = {fault.source_id: frac} 
        #
        #
        if frac is not None and frac > 0.5:
            fault.id_intersecting_area = area_source_ids_list[0]
            
print('Faults intersecting the area source %s\n' % (area_source_ids_list[0]))     
#print(p)

model.sources[area_source_ids_list[0]] = area_source
oqtkp.models[model_id] = model
oqtkp.save()

165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.8767150464
167.411822338 -10.9818267206 253711.3452078026 -1596482.551688694
166.851597558 -9.87693895959 165055.84675372133 -1426254.8835567723
163.126413207 -10.1773570524 -405212.27422930324 -1467711.898740291
164.117146564 -9.01432224237 -250179.4949100572 -1292895.1033617053
170.294645687 -9.64351196784 688874.9067844187 -1374191.4342602948
-172.400633324 -14.0630630118 3460283.374321408 -1674642.2869322202
-172.157400982 -14.1586185748 3501854.999237324 -1680548.9986706667
-178.080690124 -13.9852540145 2568818.877696934 -1850648.5126820838
-179.667993668 -15.0745799761 2347309.7222588668 -2070964.1799458463
176.72166886 -16.1016587398 1789340.8583197994 -2324047.9656182523
173.982792156 -16.4128947288 1348004.9613911696 -2423910.797502122
173.827174162 -17.3466026959 1336716.4487533448 -2585830.332538789
175.352230508 -17.5333442893 1592300.2836642484 -259

166.663576913 -13.6878744223 142269.19757578717 -2030206.088457229
166.478114371 -13.4405910338 112397.95940505575 -1990236.7480039839
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.8767150464
167.411822338 -10.9818267206 253711.3452078026 -1596482.551688694
166.851597558 -9.87693895959 165055.84675372133 -1426254.8835567723
163.126413207 -10.1773570524 -405212.27422930324 -1467711.898740291
164.117146564 -9.01432224237 -250179.4949100572 -1292895.1033617053
170.294645687 -9.64351196784 688874.9067844187 -1374191.4342602948
-172.400633324 -14.0630630118 3460283.374321408 -1674642.2869322202
-172.157400982 -14.1586185748 3501854.999237324 -1680548.9986706667
-178.080690124 -13.9852540145 2568818.877696934 -1850648.5126820838
-179.667993668 -15.0745799761 2347309.7222588668 -2070964.1799458463
176.72166886 -16.1016587398 1789340.8583197994 -2

-179.667993668 -15.0745799761 2347309.7222588668 -2070964.1799458463
176.72166886 -16.1016587398 1789340.8583197994 -2324047.9656182523
173.982792156 -16.4128947288 1348004.9613911696 -2423910.797502122
173.827174162 -17.3466026959 1336716.4487533448 -2585830.332538789
175.352230508 -17.5333442893 1592300.2836642484 -2593457.790714116
176.908410453 -17.2221083003 1843128.281012777 -2510518.863982348
177.347697481 -17.1159472686 1913112.9054445152 -2483272.5993632013
178.371219601 -19.49413102 2139029.3066754434 -2875072.846328164
176.235286832 -21.6520837148 1823613.157776451 -3313726.3915156736
175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.40610713

168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096
167.691934728 -12.9737370503 303702.96728739724 -1911928.5203137896
166.663576913 -13.6878744223 142269.19757578717 -2030206.088457229
166.478114371 -13.4405910338 112397.95940505575 -1990236.7480039839
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.8767150464
167.411822338 -10.9818267206 253711.3452078026 -1596482.551688694
166.851597558 -9.87693895959 165055.84675372133 -1426254.8835567723
163.126413207 -10.1773570524 -405212.27422930324 -1467711.898740291
164.117146564 -9.01432224237 -250179.4949100572 -1292895.1033617053
170.294645687 -9.64351196784 688874.9067844187 -1374191.4342602948
-172.400633324 -14.0630630118 3460283.374321408 -1674642.2869322202
-172.157400982 -14.1586185748 3501854.999237324 -1680

173.827174162 -17.3466026959 1336716.4487533448 -2585830.332538789
175.352230508 -17.5333442893 1592300.2836642484 -2593457.790714116
176.908410453 -17.2221083003 1843128.281012777 -2510518.863982348
177.347697481 -17.1159472686 1913112.9054445152 -2483272.5993632013
178.371219601 -19.49413102 2139029.3066754434 -2875072.846328164
176.235286832 -21.6520837148 1823613.157776451 -3313726.3915156736
175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096


...

173.827174162 -17.3466026959 1336716.4487533448 -2585830.332538789
175.352230508 -17.5333442893 1592300.2836642484 -2593457.790714116
176.908410453 -17.2221083003 1843128.281012777 -2510518.863982348
177.347697481 -17.1159472686 1913112.9054445152 -2483272.5993632013
178.371219601 -19.49413102 2139029.3066754434 -2875072.846328164
176.235286832 -21.6520837148 1823613.157776451 -3313726.3915156736
175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096


173.982792156 -16.4128947288 1348004.9613911696 -2423910.797502122
173.827174162 -17.3466026959 1336716.4487533448 -2585830.332538789
175.352230508 -17.5333442893 1592300.2836642484 -2593457.790714116
176.908410453 -17.2221083003 1843128.281012777 -2510518.863982348
177.347697481 -17.1159472686 1913112.9054445152 -2483272.5993632013
178.371219601 -19.49413102 2139029.3066754434 -2875072.846328164
176.235286832 -21.6520837148 1823613.157776451 -3313726.3915156736
175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.320804987

175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096
167.691934728 -12.9737370503 303702.96728739724 -1911928.5203137896
166.663576913 -13.6878744223 142269.19757578717 -2030206.088457229
166.478114371 -13.4405910338 112397.95940505575 -1990236.7480039839
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.87

175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096
167.691934728 -12.9737370503 303702.96728739724 -1911928.5203137896
166.663576913 -13.6878744223 142269.19757578717 -2030206.088457229
166.478114371 -13.4405910338 112397.95940505575 -1990236.7480039839
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.87

175.908564838 -20.9725019678 1752870.9031932887 -3194804.18989404
174.788115278 -21.4237941519 1568094.9624955386 -3298417.10391629
171.46146536 -22.4323260275 1002419.2928889882 -3534292.4461538144
170.298536136 -21.9217717343 792956.4407233199 -3449594.1838756064
169.862090467 -21.3586160322 711838.5208397455 -3348136.0015089004
170.617553025 -20.6612659788 836307.1389150389 -3213120.4061071384
170.461935031 -19.7898052096 801296.3078253331 -3056159.4609299055
168.563395498 -15.4636249623 454016.54849953746 -2317476.3208049876
168.252159509 -13.689579825 395420.762005828 -2025784.2057256096
167.691934728 -12.9737370503 303702.96728739724 -1911928.5203137896
166.663576913 -13.6878744223 142269.19757578717 -2030206.088457229
166.478114371 -13.4405910338 112397.95940505575 -1990236.7480039839
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
165.77092992 -11.896331109 -0.012529120532737047 -1742852.7211288754
167.193957146 -11.5109279019 221330.03983241963 -1680162.87